## Cellular Automata Model for EA Benchmark Test 2

Import from google drive the input dataset and plot the input data

In [ ]:
from pathlib import Path

NOTEBOOK_DIR = Path.cwd()
REPO_ROOT = NOTEBOOK_DIR.parent          # 01_ea_benchmarks

TEST2_DIR = REPO_ROOT / "data" / "test2" / "raw"
OUT_DIR = REPO_ROOT / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DEM_PATH = TEST2_DIR / "test2DEM.asc"
BC_A_PATH = TEST2_DIR / "Test2A_BC.csv"
BC_B_PATH = TEST2_DIR / "Test2B_BC.csv"
REF_PATH = TEST2_DIR / "Test2output.csv"
BC_POLYLINE_SHP = TEST2_DIR / "Test2BC_polyline.shp"
ACTIVE_AREA_SHP = TEST2_DIR / "Test2ActiveArea_region.shp"


assert TEST2_DIR.exists(), TEST2_DIR
assert DEM_PATH.exists(), DEM_PATH
assert BC_A_PATH.exists(), BC_A_PATH
assert BC_B_PATH.exists(), BC_B_PATH
assert REF_PATH.exists(), REF_PATH
assert BC_POLYLINE_SHP.exists(), BC_POLYLINE_SHP
assert ACTIVE_AREA_SHP.exists(), ACTIVE_AREA_SHP

print("✅ Test 2 paths OK")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

# --- Utility: Load ASCII DEM (.asc) ---
def load_dem(filepath):
    with open(filepath, "r") as file:
        header = {}
        for _ in range(6):
            k, v = file.readline().split()
            header[k.lower()] = float(v)
        data = np.loadtxt(file)
    return header, data

# --- Load DEM ---
dem_header, dem_data = load_dem(DEM_PATH)

# --- Grid setup ---
ncols = int(dem_header["ncols"])
nrows = int(dem_header["nrows"])
xll = dem_header["xllcorner"]
yll = dem_header["yllcorner"]
cellsize = dem_header["cellsize"]

x = np.linspace(xll, xll + cellsize * (ncols - 1), ncols)
y = np.linspace(yll, yll + cellsize * (nrows - 1), nrows)
X, Y = np.meshgrid(x, y[::-1])

# --- Load shapefiles and CSVs (use TEST2_DIR) ---
bc_poly = gpd.read_file(BC_POLYLINE_SHP)
active_area = gpd.read_file(ACTIVE_AREA_SHP)

bc_A = pd.read_csv(BC_A_PATH)
bc_B = pd.read_csv(BC_B_PATH)
outputs = pd.read_csv(REF_PATH)

In [ ]:
# ================================================================
# === Plot DEM + active area + boundary + output points ===
# ================================================================
plt.figure(figsize=(12, 6))
plt.pcolormesh(X, Y, dem_data, shading='auto', cmap='terrain')
plt.colorbar(label='Elevation (m)')

# Overlay shapefile geometries
active_area.boundary.plot(ax=plt.gca(), color='black', linewidth=1.0, label='Active Area')
bc_poly.plot(ax=plt.gca(), color='red', linewidth=1.5, label='Boundary Condition Lines')

# Output points
plt.scatter(outputs['X'], outputs['Y'], color='blue', label='Output Points', zorder=5)

plt.title("EA2D Test 2 — DEM with Boundaries and Output Points")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ================================================================
# === Plot Hydrographs for BC-A and BC-B ===
# ================================================================
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

axes[0].plot(bc_A.iloc[:,0]/60, bc_A.iloc[:,1], color='green')
axes[0].set_title("Hydrograph A — Inflow vs. Time")
axes[0].set_xlabel("Time (hours)")
axes[0].set_ylabel("Inflow (m³/s)")
axes[0].grid(True)

axes[1].plot(bc_B.iloc[:,0]/60, bc_B.iloc[:,1], color='orange')
axes[1].set_title("Hydrograph B — Inflow vs. Time")
axes[1].set_xlabel("Time (hours)")
axes[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# ================================================================
# EA2D Test 2 — DEM Preprocessing (20 m grid) for CA Flood Model
# ================================================================

from scipy.spatial import cKDTree

TARGET_DX = 20.0  # Target grid resolution (m)

# -----------------------------
# DEM reading and utilities
# -----------------------------
def load_dem_asc(filepath):
    """Read ESRI ASCII grid → (header, data)."""
    with open(filepath, "r") as f:
        hdr = {}
        for _ in range(6):
            k, v = f.readline().split()
            hdr[k.lower()] = float(v)
        data = np.loadtxt(f)
    data[data == -9999] = np.nan
    return hdr, data

def downsample_mean_nan(data, factor):
    """Downsample by integer factor (NaN safe)."""
    nrows_new = data.shape[0] // factor
    ncols_new = data.shape[1] // factor
    trimmed = data[:nrows_new * factor, :ncols_new * factor]
    blocks = trimmed.reshape(nrows_new, factor, ncols_new, factor)
    return np.nanmean(blocks, axis=(1, 3))

def make_grid_coords(xll, yll, dx, nx, ny):
    """Generate coordinate grids (centers)."""
    x = np.linspace(xll, xll + dx*(nx-1), nx)
    y = np.linspace(yll, yll + dx*(ny-1), ny)
    X, Y = np.meshgrid(x, y[::-1])  # reverse y for plotting
    return X, Y

# -----------------------------
# Polyline → grid mapping helper
# -----------------------------
def grid_cells_along_polyline(poly_xy, grid_xy, dx, ny, nx, buffer=None):
    """Return all DEM grid cells intersecting the inflow polyline (not just vertices)."""
    if buffer is None:
        buffer = dx * 0.6  # ~one cell thickness around line
    chosen = np.zeros(len(grid_xy), dtype=bool)
    for k in range(len(poly_xy) - 1):
        p0, p1 = poly_xy[k], poly_xy[k + 1]
        seg = p1 - p0
        L = np.linalg.norm(seg)
        if L < 1e-9:
            continue
        dirv = seg / L
        v = grid_xy - p0
        t = np.clip((v @ dirv) / L, 0.0, 1.0)
        closest = p0 + np.outer(t, seg)
        dist = np.linalg.norm(grid_xy - closest, axis=1)
        chosen |= (dist <= buffer)
    idx = np.where(chosen)[0]
    return np.unravel_index(idx, (ny, nx))

# -----------------------------
# Main preprocessing
# -----------------------------
print("🔹 Loading DEM...")
hdr, Z_src = load_dem_asc(DEM_PATH)
cs = hdr["cellsize"]
factor = int(round(TARGET_DX / cs))
print(f"Original DEM cellsize = {cs} m → Downsample factor = {factor}")

# Downsample to 20 m grid
Z20 = downsample_mean_nan(Z_src, factor)
ny, nx = Z20.shape
X20, Y20 = make_grid_coords(hdr["xllcorner"], hdr["yllcorner"], TARGET_DX, nx, ny)
terrain_grid = np.dstack((X20, Y20, Z20))

# -----------------------------
# Load shapefiles and output CSV
# -----------------------------
print("🔹 Loading shapefiles and output points...")
bc_line = gpd.read_file(BC_POLYLINE_SHP)
active_area = gpd.read_file(ACTIVE_AREA_SHP)
outputs_df = pd.read_csv(REF_PATH)

# Ensure consistent CRS (just in case)
if bc_line.crs and active_area.crs and bc_line.crs != active_area.crs:
    bc_line = bc_line.to_crs(active_area.crs)

# -----------------------------
# Map output points to grid
# -----------------------------
grid_xy = np.column_stack((X20.ravel(), Y20.ravel()))
tree = cKDTree(grid_xy)
out_xy = outputs_df[["X", "Y"]].to_numpy(float)
_, idx_out = tree.query(out_xy)
out_elev = Z20.ravel()[idx_out]
output_points = np.column_stack((out_xy, out_elev))

# -----------------------------
# Map inflow polyline to grid (improved)
# -----------------------------
poly_xy = np.concatenate([np.array(geom.coords) for geom in bc_line.geometry])
in_i, in_j = grid_cells_along_polyline(poly_xy, grid_xy, TARGET_DX, ny, nx, buffer=TARGET_DX*0.6)
flat_idx = np.ravel_multi_index((in_i, in_j), (ny, nx))
flat_idx = np.unique(flat_idx)
inflow_points = np.column_stack((grid_xy[flat_idx], Z20.ravel()[flat_idx]))

print(f"✅ Inflow line covers {len(inflow_points)} grid cells.")

# -----------------------------
# Save arrays
# -----------------------------
np.save(OUT_DIR/ "terrain_grid_20m.npy", terrain_grid)
np.save(OUT_DIR/ "output_points.npy", output_points)
np.save(OUT_DIR/ "inflow_points.npy", inflow_points)

print("\n✅ Saved processed arrays to:")
print("  terrain_grid_20m.npy", terrain_grid.shape)
print("  output_points.npy    ", output_points.shape)
print("  inflow_points.npy    ", inflow_points.shape)

# -----------------------------
# Visualization
# -----------------------------
plt.figure(figsize=(10, 6))
pc = plt.pcolormesh(X20, Y20, np.ma.masked_invalid(Z20), shading='auto', cmap='terrain')
plt.colorbar(pc, label="Elevation (m)")

# Overlay polygons and inflow
active_area.boundary.plot(ax=plt.gca(), color='black', linewidth=1.0, label='Active Area')
bc_line.plot(ax=plt.gca(), color='red', linewidth=1.5, label='Inflow Boundary')

# Scatter inflow + output points
plt.scatter(inflow_points[:,0], inflow_points[:,1], c='cyan', label='Inflow Cells', zorder=3)
plt.scatter(output_points[:,0], output_points[:,1], c='blue', label='Output Points', zorder=3)

plt.title("EA2D Test 2 — 20 m Terrain with Inflow & Output Points")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.legend()
plt.axis('equal')
plt.tight_layout()
plt.show()


## Test 2A

In [ ]:
# ================================================================
# EA2D Test 2A — Cellular Automata 2D Flood Model 
# ================================================================
# Inputs (from preprocessing step in Drive):
#   terrain_grid_20m.npy
#   inflow_points.npy
#   output_points.npy
#   Test2A_BC.csv
#
# Outputs (stored in /PreparedTerrainData_Test2/Results_Test2A):
#   - Depth snapshots every 4 h
#   - 16 hydrographs (PNG)
#   - Final depth map (PNG)
# ================================================================

import os, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

plt.rcParams.update({
    "figure.figsize": (10, 6),
    "axes.grid": True,
    "font.size": 10
})

# ================================================================
# Parameters
# ================================================================
g = 9.81
n_manning = 0.03
theta = 0.7
CFL = 1
DT_MIN, DT_MAX = 0.5, 15.0
depth_threshold = 0.01
LOG_DT = 300.0                # 5 min
t_end = 48 * 3600.0           # 48 h
snapshot_interval = 4 * 3600  # 4 h

# ================================================================
# Load inputs
# ================================================================
print("🔹 Loading preprocessed arrays...")
terrain = np.load(OUT_DIR / "terrain_grid_20m.npy")
outputs = np.load(OUT_DIR / "output_points.npy")
inflow_vertices = np.load(OUT_DIR / "inflow_points.npy")
df_bc = pd.read_csv(BC_A_PATH)   # or BC_PATH if single BC

X, Y, Z = terrain[:, :, 0], terrain[:, :, 1], terrain[:, :, 2]
ny, nx = Z.shape
dx = float(np.median(np.diff(np.unique(X[0, :]))))
A = dx * dx
grid_xy = np.column_stack((X.ravel(), Y.ravel()))

# Map outputs to grid
tree = cKDTree(grid_xy)
_, idx_out = tree.query(outputs[:, :2])
out_i, out_j = np.unravel_index(idx_out, (ny, nx))

# -----------------------------
# Inflow line to grid
# -----------------------------
def grid_cells_along_polyline(poly_xy, grid_xy, dx, ny, nx, buffer=None):
    if buffer is None: buffer = dx * 0.6
    chosen = np.zeros(len(grid_xy), dtype=bool)
    for k in range(len(poly_xy) - 1):
        p0, p1 = poly_xy[k], poly_xy[k + 1]
        seg = p1 - p0
        L = np.linalg.norm(seg)
        if L < 1e-9: continue
        dirv = seg / L
        v = grid_xy - p0
        t = np.clip((v @ dirv) / L, 0.0, 1.0)
        closest = p0 + np.outer(t, seg)
        dist = np.linalg.norm(grid_xy - closest, axis=1)
        chosen |= (dist <= buffer)
    idx = np.where(chosen)[0]
    return np.unravel_index(idx, (ny, nx))

poly_xy = inflow_vertices[:, :2]
in_i, in_j = grid_cells_along_polyline(poly_xy, grid_xy, dx, ny, nx)
print(f"✅ Inflow line mapped → {len(in_i)} grid cells (~{len(in_i)*dx:.0f} m)")

# -----------------------------
# Boundary hydrograph Q(t)
# -----------------------------
tcol = next(c for c in df_bc.columns if "time" in c.lower())
qcol = next(c for c in df_bc.columns if any(k in c.lower() for k in ("flow","discharge","inflow","q","cumecs")))
t_bc = df_bc[tcol].to_numpy(float)
if t_bc.max() <= 48: t_bc *= 3600.0
elif t_bc.max() <= 3000: t_bc *= 60.0
Q_bc = df_bc[qcol].to_numpy(float)

def Qin_at(t):
    if t <= t_bc[0]: return Q_bc[0]
    if t >= t_bc[-1]: return Q_bc[-1]
    i = np.searchsorted(t_bc, t)
    t0, t1 = t_bc[i-1], t_bc[i]
    q0, q1 = Q_bc[i-1], Q_bc[i]
    return q0 + (q1 - q0) * (t - t0) / (t1 - t0)

# ================================================================
# Simulation setup
# ================================================================
d = np.zeros_like(Z)
WL = Z.copy()
t, step = 0.0, 0
Z_up = np.vstack([Z[0:1, :], Z[:-1, :]])
Z_dn = np.vstack([Z[1:, :],  Z[-1:, :]])
Z_lt = np.hstack([Z[:, 0:1], Z[:, :-1]])
Z_rt = np.hstack([Z[:, 1:],  Z[:, -1:]])

# Helper functions
def vcap(d_if, slope):
    slope = np.maximum(slope, 1e-4)
    d_if = np.maximum(d_if, 0.0)
    v_m = (1.0/n_manning) * np.where(d_if>0, d_if**(2/3), 0.0) * np.sqrt(slope)
    v_c = np.sqrt(g * d_if)
    return np.minimum(v_m, v_c)

def compute_dt(WL):
    WL_up = np.vstack([WL[0:1, :], WL[:-1, :]])
    WL_dn = np.vstack([WL[1:, :],  WL[-1:, :]])
    WL_lt = np.hstack([WL[:, 0:1], WL[:, :-1]])
    WL_rt = np.hstack([WL[:, 1:], WL[:, -1:]])
    d_if_up = np.maximum(np.maximum(WL, WL_up) - np.maximum(Z, Z_up), 0.0)
    d_if_dn = np.maximum(np.maximum(WL, WL_dn) - np.maximum(Z, Z_dn), 0.0)
    d_if_lt = np.maximum(np.maximum(WL, WL_lt) - np.maximum(Z, Z_lt), 0.0)
    d_if_rt = np.maximum(np.maximum(WL, WL_rt) - np.maximum(Z, Z_rt), 0.0)
    dWL_up = np.maximum(WL - WL_up, 0.0) / dx
    dWL_dn = np.maximum(WL - WL_dn, 0.0) / dx
    dWL_lt = np.maximum(WL - WL_lt, 0.0) / dx
    dWL_rt = np.maximum(WL - WL_rt, 0.0) / dx
    v_up = vcap(d_if_up, dWL_up)
    v_dn = vcap(d_if_dn, dWL_dn)
    v_lt = vcap(d_if_lt, dWL_lt)
    v_rt = vcap(d_if_rt, dWL_rt)
    vmax = float(max(v_up.max(), v_dn.max(), v_lt.max(), v_rt.max(), 1e-10))
    return float(np.clip(CFL * dx / vmax, DT_MIN, DT_MAX))

# ================================================================
# Run simulation
# ================================================================
print("\n🚀 Starting 48 h CA flood simulation...")
times, WL_log, snapshots = [], [], []
next_log, next_snap = 0.0, snapshot_interval
total_in, last_snap_time = 0.0, 0.0
start = time.time()

while t < t_end:
    wet_frac = np.count_nonzero(d > depth_threshold)/d.size
    dt = min(DT_MAX if wet_frac < 0.002 else compute_dt(WL), t_end - t)

    # --- inflow
    Qin = Qin_at(t)
    if Qin > 0 and len(in_i) > 0:
        vol = Qin * dt
        total_in += vol
        d[in_i, in_j] += (vol / len(in_i)) / A
        WL[in_i, in_j] = Z[in_i, in_j] + d[in_i, in_j]

    # --- neighbor W/L
    WL_up = np.vstack([WL[0:1, :], WL[:-1, :]])
    WL_dn = np.vstack([WL[1:, :],  WL[-1:, :]])
    WL_lt = np.hstack([WL[:, 0:1], WL[:, :-1]])
    WL_rt = np.hstack([WL[:, 1:],  WL[:, -1:]])

    d_if_up = np.maximum(np.maximum(WL, WL_up) - np.maximum(Z, Z_up), 0.0)
    d_if_dn = np.maximum(np.maximum(WL, WL_dn) - np.maximum(Z, Z_dn), 0.0)
    d_if_lt = np.maximum(np.maximum(WL, WL_lt) - np.maximum(Z, Z_lt), 0.0)
    d_if_rt = np.maximum(np.maximum(WL, WL_rt) - np.maximum(Z, Z_rt), 0.0)

    dWL_up = np.maximum(WL - WL_up, 0.0) / dx
    dWL_dn = np.maximum(WL - WL_dn, 0.0) / dx
    dWL_lt = np.maximum(WL - WL_lt, 0.0) / dx
    dWL_rt = np.maximum(WL - WL_rt, 0.0) / dx

    v_up = vcap(d_if_up, dWL_up)
    v_dn = vcap(d_if_dn, dWL_dn)
    v_lt = vcap(d_if_lt, dWL_lt)
    v_rt = vcap(d_if_rt, dWL_rt)

    # --- fluxes
    F_up = v_up * d_if_up * dx * dt
    F_dn = v_dn * d_if_dn * dx * dt
    F_lt = v_lt * d_if_lt * dx * dt
    F_rt = v_rt * d_if_rt * dx * dt

    # --- outflow cap
    V = d * A
    Fsum = F_up + F_dn + F_lt + F_rt
    mask = Fsum > V
    if mask.any():
        scale = np.ones_like(V)
        scale[mask] = V[mask] / Fsum[mask]
        F_up *= scale; F_dn *= scale; F_lt *= scale; F_rt *= scale

    # --- symmetric exchange
    dV = np.zeros_like(V)
    dV -= F_up; dV[1:, :]  += F_dn[:-1, :]
    dV -= F_dn; dV[:-1, :] += F_up[1:, :]
    dV -= F_lt; dV[:, 1:]  += F_rt[:, :-1]
    dV -= F_rt; dV[:, :-1] += F_lt[:, 1:]

    # --- update
    d = np.maximum(d + theta * (dV / A), 0.0)
    WL = Z + d

    # --- time advance & logging
    t += dt; step += 1
    while t >= next_log:
        times.append(next_log)
        WL_log.append(WL[out_i, out_j].copy())
        next_log += LOG_DT
    if t >= next_snap:
        snapshots.append((t, d.copy()))
        next_snap += snapshot_interval
    if step % 500 == 0:
        print(f"t={t/3600:6.2f} h | step={step:6d} | max depth={d.max():.3f} m")

print(f"\n✅ Simulation complete in {(time.time()-start)/60:.1f} min")
stored = float(np.sum(d*A))
print(f"Total inflow={total_in:.0f} m³ | Stored={stored:.0f} m³ | Balance error={stored-total_in:+.0f} m³")



In [ ]:
# ================================================================
# Results & Plots
# ================================================================
times_h = np.array(times)/3600.0
WL_out = np.vstack(WL_log)

# --- Hydrographs
fig, axes = plt.subplots(4, 4, figsize=(14, 10), sharex=True, sharey=True, constrained_layout=True)
axes = axes.ravel()
for k in range(min(16, WL_out.shape[1])):
    ax = axes[k]
    ax.plot(times_h, WL_out[:, k], lw=1.2)
    ax.set_title(f"P{k+1} ({outputs[k,0]:.0f},{outputs[k,1]:.0f})", fontsize=8)
    if k // 4 == 3: ax.set_xlabel("Time (h)")
    if k % 4 == 0: ax.set_ylabel("WL (m)")
for ax in axes[WL_out.shape[1]:]:
    ax.axis("off")
plt.suptitle("EA2D Test 2A — Water-level hydrographs (48 h)", fontsize=14)
plt.savefig(OUT_DIR/ "Hydrographs_Test2A.png", dpi=300)
plt.show()

# --- Final depth map
plt.figure(figsize=(10,7))
pc = plt.pcolormesh(X, Y, d, shading="auto", cmap="Blues", vmin=0, vmax=max(0.02, float(np.nanmax(d))))
plt.colorbar(pc, label="Depth (m)")
plt.contour(X, Y, Z, colors='gray', alpha=0.3, linewidths=0.5)
plt.scatter(X[in_i, in_j], Y[in_i, in_j], c="red", s=28, marker="s", label="Inflow cells")
plt.scatter(outputs[:,0], outputs[:,1], c="k", marker="x", s=36, label="Output points")
plt.title("EA2D Test 2A — Final flood depth (48 h)")
plt.xlabel("X (m)"); plt.ylabel("Y (m)")
plt.axis("equal"); plt.legend(); plt.tight_layout()
plt.savefig(OUT_DIR/ "FinalDepth_48h.png", dpi=300)
plt.show()

# --- Snapshots every 4 h
for t_snap, d_snap in snapshots:
    plt.figure(figsize=(8,6))
    pc = plt.pcolormesh(X, Y, d_snap, shading="auto", cmap="Blues", vmin=0, vmax=np.nanmax(d))
    plt.colorbar(pc, label="Depth (m)")
    plt.contour(X, Y, Z, colors='gray', alpha=0.3, linewidths=0.5)
    plt.title(f"Water depth snapshot — t = {t_snap/3600:.1f} h")
    plt.axis("equal"); plt.tight_layout()
    fname = f"Snapshot_{int(t_snap/3600):02d}h.png"
    plt.savefig(OUT_DIR/ fname, dpi=200)
    plt.close()

print(f"\n📂 All results saved in {OUT_DIR}")


## Test 2B

In [ ]:
# ================================================================
# EA2D Test 2B — Cellular Automata 2D Flood Model (Final Fixed)
# ================================================================
# Fixes:
#   ✅ Use preprocessed inflow grid cells directly (no remapping)
#   ✅ Preserve inflow continuity (no zero-reset between timesteps)
#   ✅ Maintain ponding layer and avoid dry–wet oscillations
# ================================================================

import os, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

plt.rcParams.update({
    "figure.figsize": (10, 6),
    "axes.grid": True,
    "font.size": 10
})

# ================================================================
# Parameters
# ================================================================
g = 9.81
n_manning = 0.03
theta = 0.7
CFL = 2
DT_MIN, DT_MAX = 0.5, 15.0
depth_threshold = 0.01
min_depth = 0.005            # residual ponding depth (m)
LOG_DT = 300.0               # logging every 5 min
snapshot_interval = 4 * 3600 # 4 h
t_end = 96 * 3600.0          # 96 h

# ================================================================
# Load inputs
# ================================================================
print("🔹 Loading preprocessed arrays...")
terrain = np.load(OUT_DIR / "terrain_grid_20m.npy")
outputs = np.load(OUT_DIR / "output_points.npy")
inflow_vertices = np.load(OUT_DIR / "inflow_points.npy")
df_bc = pd.read_csv(BC_A_PATH)   # or BC_PATH if single BC

X, Y, Z = terrain[:, :, 0], terrain[:, :, 1], terrain[:, :, 2]
ny, nx = Z.shape
dx = float(np.median(np.diff(np.unique(X[0, :]))))
A = dx * dx
grid_xy = np.column_stack((X.ravel(), Y.ravel()))
tree = cKDTree(grid_xy)

# ================================================================
# Map output and inflow cells
# ================================================================
_, idx_out = tree.query(outputs[:, :2])
out_i, out_j = np.unravel_index(idx_out, (ny, nx))

# ✅ Use preprocessed inflow grid cells directly (no remapping)
_, idx_inflow = tree.query(inflow_points[:, :2])
in_i, in_j = np.unravel_index(idx_inflow, (ny, nx))
print(f"✅ Using preprocessed inflow points → {len(in_i)} grid cells")

# ================================================================
# Boundary condition
# ================================================================
tcol = next(c for c in df_bc.columns if "time" in c.lower())
qcol = next(c for c in df_bc.columns if any(k in c.lower() for k in ("flow","discharge","inflow","q","cumecs")))
t_bc = df_bc[tcol].to_numpy(float)
if t_bc.max() <= 48: t_bc *= 3600.0
elif t_bc.max() <= 3000: t_bc *= 60.0
Q_bc = df_bc[qcol].to_numpy(float)

def Qin_at(t):
    if t <= t_bc[0]: return Q_bc[0]
    if t >= t_bc[-1]: return Q_bc[-1]
    i = np.searchsorted(t_bc, t)
    t0, t1 = t_bc[i-1], t_bc[i]
    q0, q1 = Q_bc[i-1], Q_bc[i]
    return q0 + (q1 - q0)*(t - t0)/(t1 - t0)

# ================================================================
# Simulation setup
# ================================================================
d = np.zeros_like(Z)
WL = Z.copy()
t, step = 0.0, 0
Z_up = np.vstack([Z[0:1, :], Z[:-1, :]])
Z_dn = np.vstack([Z[1:, :],  Z[-1:, :]])
Z_lt = np.hstack([Z[:, 0:1], Z[:, :-1]])
Z_rt = np.hstack([Z[:, 1:],  Z[:, -1:]])

def vcap(d_if, slope):
    slope = np.maximum(slope, 1e-4)
    d_if = np.maximum(d_if, 0.0)
    v_m = (1.0/n_manning) * np.where(d_if>0, d_if**(2/3), 0.0) * np.sqrt(slope)
    v_c = np.sqrt(g * d_if)
    return np.minimum(v_m, v_c)

def compute_dt(WL):
    WL_up = np.vstack([WL[0:1, :], WL[:-1, :]])
    WL_dn = np.vstack([WL[1:, :],  WL[-1:, :]])
    WL_lt = np.hstack([WL[:, 0:1], WL[:, :-1]])
    WL_rt = np.hstack([WL[:, 1:],  WL[:, -1:]])
    d_if_up = np.maximum(np.maximum(WL, WL_up) - np.maximum(Z, Z_up), 0.0)
    d_if_dn = np.maximum(np.maximum(WL, WL_dn) - np.maximum(Z, Z_dn), 0.0)
    d_if_lt = np.maximum(np.maximum(WL, WL_lt) - np.maximum(Z, Z_lt), 0.0)
    d_if_rt = np.maximum(np.maximum(WL, WL_rt) - np.maximum(Z, Z_rt), 0.0)
    dWL_up = np.maximum(WL - WL_up, 0.0) / dx
    dWL_dn = np.maximum(WL - WL_dn, 0.0) / dx
    dWL_lt = np.maximum(WL - WL_lt, 0.0) / dx
    dWL_rt = np.maximum(WL - WL_rt, 0.0) / dx
    v_up = vcap(d_if_up, dWL_up)
    v_dn = vcap(d_if_dn, dWL_dn)
    v_lt = vcap(d_if_lt, dWL_lt)
    v_rt = vcap(d_if_rt, dWL_rt)
    vmax = float(max(v_up.max(), v_dn.max(), v_lt.max(), v_rt.max(), 1e-10))
    return float(np.clip(0.1*CFL*dx/vmax, DT_MIN, DT_MAX))

# ================================================================
# Main simulation loop
# ================================================================
print("\n🚀 Starting 96 h CA flood simulation...")
times, WL_log, snapshots = [], [], []
next_log, next_snap = 0.0, snapshot_interval
total_in = 0.0
start = time.time()

while t < t_end:
    wet_frac = np.count_nonzero(d > depth_threshold)/d.size
    dt = min(DT_MAX if wet_frac < 0.002 else compute_dt(WL), t_end - t)

    # --- Inflow (buffered and accumulated)
    Qin = Qin_at(t)
    if Qin > 0 and len(in_i) > 0:
        vol = Qin * dt
        total_in += vol
        d_add = (vol / len(in_i)) / A
        d[in_i, in_j] = np.maximum(d[in_i, in_j] + d_add, min_depth)
        WL[in_i, in_j] = Z[in_i, in_j] + d[in_i, in_j]

    # --- Neighbouring water levels
    WL_up = np.vstack([WL[0:1, :], WL[:-1, :]])
    WL_dn = np.vstack([WL[1:, :],  WL[-1:, :]])
    WL_lt = np.hstack([WL[:, 0:1], WL[:, :-1]])
    WL_rt = np.hstack([WL[:, 1:],  WL[:, -1:]])

    d_if_up = np.maximum(np.maximum(WL, WL_up) - np.maximum(Z, Z_up), 0.0)
    d_if_dn = np.maximum(np.maximum(WL, WL_dn) - np.maximum(Z, Z_dn), 0.0)
    d_if_lt = np.maximum(np.maximum(WL, WL_lt) - np.maximum(Z, Z_lt), 0.0)
    d_if_rt = np.maximum(np.maximum(WL, WL_rt) - np.maximum(Z, Z_rt), 0.0)

    dWL_up = np.maximum(WL - WL_up, 0.0) / dx
    dWL_dn = np.maximum(WL - WL_dn, 0.0) / dx
    dWL_lt = np.maximum(WL - WL_lt, 0.0) / dx
    dWL_rt = np.maximum(WL - WL_rt, 0.0) / dx

    v_up = vcap(d_if_up, dWL_up)
    v_dn = vcap(d_if_dn, dWL_dn)
    v_lt = vcap(d_if_lt, dWL_lt)
    v_rt = vcap(d_if_rt, dWL_rt)

    # --- Fluxes
    F_up = v_up * d_if_up * dx * dt
    F_dn = v_dn * d_if_dn * dx * dt
    F_lt = v_lt * d_if_lt * dx * dt
    F_rt = v_rt * d_if_rt * dx * dt

    # --- Prevent draining of ponding cells
    mask_pond = d > min_depth
    F_up *= mask_pond
    F_dn *= mask_pond
    F_lt *= mask_pond
    F_rt *= mask_pond

    # --- Outflow scaling
    V = d * A
    Fsum = F_up + F_dn + F_lt + F_rt
    mask = Fsum > V
    if mask.any():
        scale = np.ones_like(V)
        scale[mask] = V[mask] / Fsum[mask]
        F_up *= scale; F_dn *= scale; F_lt *= scale; F_rt *= scale

    # --- Symmetric exchange
    dV = np.zeros_like(V)
    dV -= F_up; dV[1:, :]  += F_dn[:-1, :]
    dV -= F_dn; dV[:-1, :] += F_up[1:, :]
    dV -= F_lt; dV[:, 1:]  += F_rt[:, :-1]
    dV -= F_rt; dV[:, :-1] += F_lt[:, 1:]

    # --- Update
    d = np.maximum(d + theta*(dV/A), 0.0)
    WL = Z + d

    # --- Logging
    t += dt; step += 1
    while t >= next_log:
        times.append(next_log)
        WL_log.append(WL[out_i, out_j].copy())
        next_log += LOG_DT
    if t >= next_snap:
        snapshots.append((t, d.copy()))
        next_snap += snapshot_interval
    if step % 500 == 0:
        print(f"t={t/3600:6.2f} h | step={step:6d} | max depth={d.max():.3f} m")

print(f"\n✅ Simulation complete in {(time.time()-start)/60:.1f} min")
stored = float(np.sum(d*A))
print(f"Total inflow={total_in:.0f} m³ | Stored={stored:.0f} m³ | Balance error={stored-total_in:+.0f} m³")



In [ ]:
# ================================================================
# Results and plots
# ================================================================
times_h = np.array(times)/3600.0
WL_out = np.vstack(WL_log)

# --- Hydrographs
fig, axes = plt.subplots(4, 4, figsize=(14, 10), sharex=True, sharey=True, constrained_layout=True)
axes = axes.ravel()
for k in range(min(16, WL_out.shape[1])):
    ax = axes[k]
    ax.plot(times_h, WL_out[:, k], lw=1.2)
    ax.set_title(f"P{k+1} ({outputs[k,0]:.0f},{outputs[k,1]:.0f})", fontsize=8)
    if k // 4 == 3: ax.set_xlabel("Time (h)")
    if k % 4 == 0: ax.set_ylabel("WL (m)")
for ax in axes[WL_out.shape[1]:]:
    ax.axis("off")
plt.suptitle("EA2D Test 2B — Water-level hydrographs (96 h)", fontsize=14)
plt.savefig(OUT_DIR/ "Hydrographs_Test2B.png", dpi=300)
plt.show()

# --- Final depth map
plt.figure(figsize=(10,7))
pc = plt.pcolormesh(X, Y, d, shading="auto", cmap="Blues", vmin=0, vmax=np.nanmax(d))
plt.colorbar(pc, label="Depth (m)")
plt.contour(X, Y, Z, colors='gray', alpha=0.3, linewidths=0.5)
plt.scatter(X[in_i, in_j], Y[in_i, in_j], c="red", s=28, marker="s", label="Inflow cells")
plt.scatter(outputs[:,0], outputs[:,1], c="k", marker="x", s=36, label="Output points")
plt.title("EA2D Test 2B — Final flood depth (96 h)")
plt.xlabel("X (m)"); plt.ylabel("Y (m)")
plt.axis("equal"); plt.legend(); plt.tight_layout()
plt.savefig(OUT_DIR/ "FinalDepth_96h.png", dpi=300)
plt.show()

print(f"\n📂 Results saved in: {OUT_DIR}")
